In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,093 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [1,073 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 https://developer.download.nvid

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-29 00:49:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.45MB/s    in 0.6s    

2022-08-29 00:49:17 (1.45 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in CSV from S3 into a DataFrame
url = "https://myopia.s3.amazonaws.com/Copy_of_myopia.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("Copy_of_myopia.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+---+---------+------+---+------+------+-----+-----+-----+-----+-------+------+------+-------+----+---------+-----+-----+
| ID|STUDYYEAR|MYOPIC|AGE|GENDER| SPHEQ|   AL|  ACD|   LT|  VCD|SPORTHR|READHR|COMPHR|STUDYHR|TVHR|DIOPTERHR|MOMMY|DADMY|
+---+---------+------+---+------+------+-----+-----+-----+-----+-------+------+------+-------+----+---------+-----+-----+
|  1|     1992|     1|  6|     1|-0.052|21.89| 3.69|3.498| 14.7|     45|     8|     0|      0|  10|       34|    1|    1|
|  2|     1995|     0|  6|     1| 0.608|22.38|3.702|3.392|15.29|      4|     0|     1|      1|   7|       12|    1|    1|
|  3|     1991|     0|  6|     1| 1.179|22.49|3.462|3.514|15.52|     14|     0|     2|      0|  10|       14|    0|    0|
|  4|     1990|     1|  6|     1| 0.525| 22.2|3.862|3.612|14.73|     18|    11|     0|      0|   4|       37|    0|    1|
|  5|     1995|     0|  5|     0| 0.697|23.29|3.676|3.454|16.16|     14|     0|     0|      0|   4|        4|    1|    0|
|  6|     1995|     0|  

Drop duplicates and incomplete rows

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

618
618
618


Examine the schema

In [ ]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- STUDYYEAR: integer (nullable = true)
 |-- MYOPIC: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- GENDER: integer (nullable = true)
 |-- SPHEQ: double (nullable = true)
 |-- AL: double (nullable = true)
 |-- ACD: double (nullable = true)
 |-- LT: double (nullable = true)
 |-- VCD: double (nullable = true)
 |-- SPORTHR: integer (nullable = true)
 |-- READHR: integer (nullable = true)
 |-- COMPHR: integer (nullable = true)
 |-- STUDYHR: integer (nullable = true)
 |-- TVHR: integer (nullable = true)
 |-- DIOPTERHR: integer (nullable = true)
 |-- MOMMY: integer (nullable = true)
 |-- DADMY: integer (nullable = true)



In [ ]:
#save dataframe into json 
df.write.json("users_json.json")

Write DataFrame to RDS

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>/postgres"
config = {"user":"postgres",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

df.write.jdbc(url=jdbc_url, table='myopia_data', mode=mode, properties=config)